In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics
import warnings
import datetime
warnings.filterwarnings("ignore")
import os
print(os.listdir("data"))

# Any results you write to the current directory are saved as output.

['merchants.csv', 'new_merchant_transactions.csv', 'Data_Dictionary.xlsx', 'test.csv', 'train.csv', 'historical_transactions.csv', 'sample_submission.csv']


In [3]:
df_train = pd.read_csv("data/train.csv", parse_dates=["first_active_month"])
df_test = pd.read_csv("data/test.csv", parse_dates=["first_active_month"])
print("{} observations and {} features in train set.".format(df_train.shape[0],df_train.shape[1]))
print("{} observations and {} features in test set.".format(df_test.shape[0],df_test.shape[1]))

201917 observations and 6 features in train set.
123623 observations and 5 features in test set.


In [4]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [5]:
df_train["month"] = df_train["first_active_month"].dt.month
df_test["month"] = df_test["first_active_month"].dt.month

df_train["year"] = df_train["first_active_month"].dt.year
df_test["year"] = df_test["first_active_month"].dt.year

df_train['elapsed_time'] = (datetime.date(2018, 2, 1) - df_train['first_active_month'].dt.date).dt.days
df_test['elapsed_time'] = (datetime.date(2018, 2, 1) - df_test['first_active_month'].dt.date).dt.days
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,month,year,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,6,2017,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,1,2017,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,8,2016,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,9,2017,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,11,2017,92


In [17]:
df_train = pd.get_dummies(df_train, columns=['feature_1', 'feature_2'])
df_test = pd.get_dummies(df_test, columns=['feature_1', 'feature_2'])
df_train.head()

,first_active_month,card_id,feature_3,target,month,year,elapsed_time,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3
0,2017-06-01,C_ID_92a2005557,1,-0.820283,6,2017,245,0,0,0,0,1,0,1,0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,1,2017,396,0,0,0,1,0,1,0,0
2,2016-08-01,C_ID_d639edf6cd,0,0.688056,8,2016,549,0,1,0,0,0,0,1,0
3,2017-09-01,C_ID_186d6a6901,0,0.142495,9,2017,153,0,0,0,1,0,0,0,1
4,2017-11-01,C_ID_cdbd2c0db2,0,-0.159749,11,2017,92,1,0,0,0,0,0,0,1


In [12]:
df_hist_trans = pd.read_csv("data/historical_transactions.csv")

In [13]:
df_hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [18]:
df_hist_trans = pd.get_dummies(df_hist_trans, columns=['category_2', 'category_3'])
df_hist_trans['authorized_flag'] = df_hist_trans['authorized_flag'].map({'Y': 1, 'N': 0})
df_hist_trans['category_1'] = df_hist_trans['category_1'].map({'Y': 1, 'N': 0})
df_hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,1.498405e+09,16,37,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,1.500121e+09,16,16,1,0,0,0,0,1,0,0
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,1.502316e+09,16,37,1,0,0,0,0,1,0,0
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,1.504347e+09,16,34,1,0,0,0,0,1,0,0
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,1.489108e+09,16,37,1,0,0,0,0,1,0,0


In [19]:
def aggregate_transactions(trans, prefix):  
    trans.loc[:, 'purchase_date'] = pd.DatetimeIndex(trans['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp],
        'month_lag': ['min', 'max']
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() 
                           for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    
    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    
    return agg_trans

In [20]:
import gc
merch_hist = aggregate_transactions(df_hist_trans, prefix='hist_')
del df_hist_trans
gc.collect()
df_train = pd.merge(df_train, merch_hist, on='card_id',how='left')
df_test = pd.merge(df_test, merch_hist, on='card_id',how='left')
del merch_hist
gc.collect()
df_train.head()

,first_active_month,card_id,feature_3,target,month,year,elapsed_time,feature_1_1,feature_1_2,feature_1_3,...,hist_purchase_amount_std,hist_category_2_5.0_mean,hist_purchase_date_ptp,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_month_lag_min,hist_month_lag_max
0,2017-06-01,C_ID_92a2005557,1,-0.820283,6,2017,245,0,0,0,...,0.212139,0.011538,0.020978,4,0.015385,1,0,0.123314,-8,0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,1,2017,396,0,0,0,...,0.384967,0.000000,0.033718,543,1.551429,10,-1,1.510777,-12,0
2,2016-08-01,C_ID_d639edf6cd,0,0.688056,8,2016,549,0,1,0,...,0.087380,0.906977,0.035636,0,0.000000,0,0,0.000000,-13,0
3,2017-09-01,C_ID_186d6a6901,0,0.142495,9,2017,153,0,0,0,...,0.261624,0.000000,0.013375,84,1.090909,3,-1,0.588974,-5,0
4,2017-11-01,C_ID_cdbd2c0db2,0,-0.159749,11,2017,92,1,0,0,...,1.352094,0.007519,0.009406,182,1.368421,12,1,1.896862,-3,0


In [9]:
df_new_trans = pd.read_csv("data/new_merchant_transactions.csv")
df_new_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [11]:
list(df_new_trans)

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id']

In [21]:
df_new_trans = pd.get_dummies(df_new_trans, columns=['category_2', 'category_3'])
df_new_trans['authorized_flag'] = df_new_trans['authorized_flag'].map({'Y': 1, 'N': 0})
df_new_trans['category_1'] = df_new_trans['category_1'].map({'Y': 1, 'N': 0})
df_new_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0
2,1,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0
3,1,C_ID_415bb3a509,-1,1,1,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,-1,8,0,0,0,0,0,0,1,0
4,1,C_ID_ef55cf8d4b,-1,1,1,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,-1,29,0,0,0,0,0,0,1,0


In [22]:
merch_new = aggregate_transactions(df_new_trans, prefix='new_')
del df_new_trans
gc.collect()
df_train = pd.merge(df_train, merch_new, on='card_id',how='left')
df_test = pd.merge(df_test, merch_new, on='card_id',how='left')
del merch_new
gc.collect()
df_train.head()

,first_active_month,card_id,feature_3,target,month,year,elapsed_time,feature_1_1,feature_1_2,feature_1_3,...,new_purchase_amount_std,new_category_2_5.0_mean,new_purchase_date_ptp,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_month_lag_min,new_month_lag_max
0,2017-06-01,C_ID_92a2005557,1,-0.820283,6,2017,245,0,0,0,...,0.135812,0.0,4742309.0,0.0,0.000000,0.0,0.0,0.000000,1.0,2.0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,1,2017,396,0,0,0,...,0.014326,0.0,4887632.0,6.0,1.000000,1.0,1.0,0.000000,1.0,2.0
2,2016-08-01,C_ID_d639edf6cd,0,0.688056,8,2016,549,0,1,0,...,NaN,1.0,0.0,0.0,0.000000,0.0,0.0,NaN,2.0,2.0
3,2017-09-01,C_ID_186d6a6901,0,0.142495,9,2017,153,0,0,0,...,0.065882,0.0,3625505.0,5.0,0.714286,1.0,-1.0,0.755929,1.0,2.0
4,2017-11-01,C_ID_cdbd2c0db2,0,-0.159749,11,2017,92,1,0,0,...,0.223821,0.0,4949682.0,35.0,0.972222,2.0,-1.0,0.376913,1.0,2.0


In [23]:
target = df_train['target']
drops = ['card_id', 'first_active_month', 'target']
use_cols = [c for c in df_train.columns if c not in drops]
features = list(df_train[use_cols].columns)
df_train[features].head()

,feature_3,month,year,elapsed_time,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,...,new_purchase_amount_std,new_category_2_5.0_mean,new_purchase_date_ptp,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_month_lag_min,new_month_lag_max
0,1,6,2017,245,0,0,0,0,1,0,...,0.135812,0.0,4742309.0,0.0,0.000000,0.0,0.0,0.000000,1.0,2.0
1,0,1,2017,396,0,0,0,1,0,1,...,0.014326,0.0,4887632.0,6.0,1.000000,1.0,1.0,0.000000,1.0,2.0
2,0,8,2016,549,0,1,0,0,0,0,...,NaN,1.0,0.0,0.0,0.000000,0.0,0.0,NaN,2.0,2.0
3,0,9,2017,153,0,0,0,1,0,0,...,0.065882,0.0,3625505.0,5.0,0.714286,1.0,-1.0,0.755929,1.0,2.0
4,0,11,2017,92,1,0,0,0,0,0,...,0.223821,0.0,4949682.0,35.0,0.972222,2.0,-1.0,0.376913,1.0,2.0


In [34]:
print(df_train[features].shape)
print(df_test[features].shape)

(201917, 67)
(123623, 66)


In [39]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))

In [37]:
from sklearn import linear_model

In [38]:
reg = linear_model.LinearRegression()

In [62]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, target.values)):
    print('-')
    print("Fold {}".format(fold_ + 1))
    reg.fit(df_train.iloc[trn_idx][features].fillna(0), target.iloc[trn_idx].fillna(0))
    val_pred = reg.predict(df_train.iloc[val_idx][features].fillna(0))
    test_pred = reg.predict(df_test[features].fillna(0))
    MSE = np.sum((target.iloc[val_idx].fillna(0) - val_pred)**2)/len(val_pred)
    print("MSE {}".format(MSE))

-
Fold 1
MSE 14.7194187763
(123623,)
-
Fold 2
MSE 14.5736844495
(123623,)
-
Fold 3
MSE 13.9409967106
(123623,)
-
Fold 4
MSE 15.5716238486
(123623,)
-
Fold 5
MSE 14.0078887405
(123623,)


In [66]:
df_sub = pd.DataFrame({"card_id":df_test["card_id"].values})
df_sub["target"] = test_pred
df_sub.to_csv("sub.csv", index=False)